In [1]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [17]:
df = pd.read_csv('output2.csv')

In [19]:
df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]

In [ ]:
df

In [21]:
df['공사종류(중분류)'].value_counts()

공사종류(중분류)
건축물          700
상하수도          62
도로            51
철도            25
기타            24
교량            20
터널            14
하천            14
환경시설          14
발전시설          12
옹벽 및 절토사면     11
항만             6
산업생산시설         4
댐              3
공원             2
생태공원           2
Name: count, dtype: int64

In [22]:
df['공종(중분류)'].value_counts()

공종(중분류)
철근콘크리트공사        290
가설공사             86
기타               53
토공사              44
해체 및 철거공사        40
기계설비공사           37
철골공사             36
관공사              31
수장공사             29
타일 및 돌공사         28
건축물 부대공사         26
미장공사             24
금속공사             23
조적공사             20
도로 및 포장공사        18
도장공사             17
철도 및 궤도공사        16
터널공사             16
전기설비공사           15
교량공사             15
건축 토공사           12
목공사              11
방수공사             10
관공사 부대공사         10
하천공사             10
창호 및 유리공사         8
지붕 및 홈통공사         7
말뚝공사              6
산업설비공사            5
항만공사              4
댐 및 제방공사          3
강구조물공사            3
프리캐스트 콘크리트공사      2
지반개량공사            2
지정공사              2
지반조사              2
조경공사              2
특수 건축물공사          1
Name: count, dtype: int64

In [23]:
df['인적사고'].value_counts()

인적사고
물체에 맞음                   136
끼임                       128
넘어짐(미끄러짐)                104
부딪힘                       96
떨어짐(2미터 미만)               82
절단, 베임                    76
넘어짐(물체에 걸림)               73
넘어짐(기타)                   61
기타                        46
떨어짐(2미터 이상 ~ 3미터 미만)      33
떨어짐(3미터 이상 ~ 5미터 미만)      27
찔림                        16
질병                        16
분류불능                      15
깔림                        13
떨어짐(5미터 이상 ~ 10미터 미만)     10
없음                         9
떨어짐(10미터 이상)               9
교통사고                       8
화상                         4
떨어짐(분류불능)                  1
질식                         1
Name: count, dtype: int64

In [5]:
grouped = df.groupby("인적사고")

In [7]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책"].astype(str)
    plan = group["재발방지대책"].dropna()
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

100%|██████████| 22/22 [00:09<00:00,  2.30it/s]


In [8]:
res

{'교통사고': '해당 사고 사례 현장 관계자 전파각 현장 작업자에게 사고 사례 전파 안전교육 실시현장 작업 시 위험성 사전 파악 후 작업 지시위험요인 발견 시 관리자에게  즉시 요청 위험요인 제거 및 안전확보 후 안전작업 실시 교육',
 '기타': '작업전 일일위험성 평가 회의TBM통한 근로자 안전교육 실시 및 거푸집지지대 하부 고정방법 개선',
 '깔림': '작업 전 동일 사고사례 전파 및 교육 점검 확인작업 전중후 충분한 작업공간 확보 위험여부 확인',
 '끼임': 'TBM시 위험요인 확인 및 안전대책교육 작업지휘자의 작업지시 및 확인 기계기구 등 작업시작 전 사용법 확인 현장 패트롤 강화',
 '넘어짐(기타)': '현장 주변 안전 유무 확인과 작업 전 안전 교육 실시',
 '넘어짐(물체에 걸림)': '작업장 정리정돈 철저 및 작업통로 확보 안전작업 수칙 미준수시 추가 교육 실시를 통한 안전의식 고취 TBM 및 안전교육 등을 통한 무리한 작업 금지 교육 실시 하도급사 작업책임자 원도급사 현장관리자 건설사업관리기술인이 현장확인을 통한 사고위험 예방',
 '넘어짐(미끄러짐)': '근로자 전도 사고 예방 특별교육 실시 및 작업장 바닥 정리 작업 실시 TBM시 전일 사고 여부 점검 철저',
 '떨어짐(10미터 이상)': '작업전 안전교육 필수 실시 및 안전보호구 지급 작업시작시 안전보호구 착용 확인 필 작업중 안전관리',
 '떨어짐(2미터 미만)': ' 현장조치 결과  현장정리 철저 공사현장 수장고 내부 습도조절을 위한 송풍기 설치 사다리 전도방 지장치 체결 철저\r\n 권고 및 향후 조치 안전사고 관련 작업자 특별교육 실시 일일 안전교육철저 재해예방기술지도업체 낙하사고 교육 및 점검',
 '떨어짐(2미터 이상 ~ 3미터 미만)': '작업시작전 작업자 안전관리교육 매일 실시 및 사고 재발방지를 위한 관리감독 철저',
 '떨어짐(3미터 이상 ~ 5미터 미만)': '관리감독자 교육 실시 및 근로자 특별 안전교육 실시 안전시설물 점검 및 설치 강화',
 '떨어짐(5미

In [9]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 0개
Range 0.1 - 0.2: 2개
Range 0.2 - 0.3: 26개
Range 0.3 - 0.4: 72개
Range 0.4 - 0.5: 133개
Range 0.5 - 0.6: 240개
Range 0.6 - 0.7: 273개
Range 0.7 - 0.8: 125개
Range 0.8 - 0.9: 12개
Range 0.9 - 1.0: 15개


In [10]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["향후조치계획"].astype(str)
    plan = group["향후조치계획"].dropna()
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

100%|██████████| 22/22 [00:09<00:00,  2.31it/s]


In [11]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 5개
Range 0.1 - 0.2: 14개
Range 0.2 - 0.3: 30개
Range 0.3 - 0.4: 70개
Range 0.4 - 0.5: 140개
Range 0.5 - 0.6: 194개
Range 0.6 - 0.7: 243개
Range 0.7 - 0.8: 158개
Range 0.8 - 0.9: 30개
Range 0.9 - 1.0: 17개


In [12]:
res

{'교통사고': '감리단 및 시공사의 안전관리 철저 지시 및 현장 안전관리 시설물 점검 및 보완',
 '기타': '현장책임자에게 관리 감독 철저 및 유사사고 재발방지를 위해 작업환경 체크 및 작업자 안전 교육 철저 요청',
 '깔림': '사고사례 전파 및 안전교육 실시 계도',
 '끼임': '작업전 안전교육 철저 실시 및 관리책임자 확인',
 '넘어짐(기타)': '현장 주변 안전 점검과 작업전 안전교육 실시하겠음',
 '넘어짐(물체에 걸림)': '현장 정리 철저 및 작업자 안전교육 실시',
 '넘어짐(미끄러짐)': '작업자 안전교육 및 현장 수시점검 등',
 '떨어짐(10미터 이상)': '난간대 시공 불량 전수조사 및 근로자 안전교육 확인 안전사고 집중 안전점검 실시',
 '떨어짐(2미터 미만)': '현장 안전관리 철저 및 사전 교육 실시',
 '떨어짐(2미터 이상 ~ 3미터 미만)': '현장 안전관리 철저 및 사전 교육 실시',
 '떨어짐(3미터 이상 ~ 5미터 미만)': '시공사감리자 행정지도 및 수시점검 실시\r\n작업지시자 상시배치 및 보호장비 착용 철저 지시',
 '떨어짐(5미터 이상 ~ 10미터 미만)': '작업전 공종별 안전교육 실시 및 근로자 개인 보호구 착용여부확인 안전시설물 점검 등의 안전점검 실시후 작업실시 및 관리감독 철저',
 '떨어짐(분류불능)': '현장책임자에게 관리 감독 철저 및 유사사고 재발방지 위해 TBM시간 활용한 안전교육 철저 현장조치 철저',
 '물체에 맞음': '작업자 교육 실시 및 유사사고 예방 철저',
 '부딪힘': '작업전 안전교육 실시 여부 확인 주기적인 안전관리실태 점검 및 지도',
 '분류불능': '지속적인 작업자 안전교육 실시 및 작업 전중후 스트레칭 관련 교육 요청',
 '없음': '사전 현장조사 및 안전관리 철저',
 '절단, 베임': '시공자 측으로 안전교육 철저 및 작업 중 지도점검 철저 요청 현장 수시점검 추진',
 '질병': '사고자 질병상태 확인 및 근로자 작업전 체조 철저',
 '질식': '공사